<a href="https://colab.research.google.com/github/codingfarm1/studing/blob/main/Pandas_08_Groupby%2CPivot_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **8. Groupby와 Pivot table**

- groupby() : 데이터를 피봇팅하여 통계량을 볼 수 있도록 도와주는 method, 데이터를 특정 조건에 맞게 전처리해 줄 때 용이합니다.

- pivot_table() : 데이터를 특정 조건에 따라 행(row)과 열(column)을 기준으로 데이터를 펼쳐서 그에 대한 통계량을 볼 때 활용합니다.

### **apply() - 함수를 적용**

- 데이터를 전처리 할 때 많이 활용함
- 좀 더 복잡한 logic을 컬럼 혹은 DataFrame에 적용하고자 할 때 사용함

In [ ]:
df['who'].value_counts()

who
man      537
woman    271
child     83
Name: count, dtype: int64

In [ ]:
# who 컬럼에 대하여 man은 남자, woman은 여자, child는 아이로 변경하고자 할 때 사용
# 함수(function)를 정의하여야 함
def transform_who(x):
    if x == 'man':
        return '남자'
    elif x == 'woman':
        return '여자'
    else:
        return '아이'
df['who'].apply(transform_who)
df['who'].apply(transform_who).value_counts()


who
남자    537
여자    271
아이     83
Name: count, dtype: int64

In [ ]:
# 함수(function) 정의
def transform_who(x):
    return x['fare'] / x['age']
df.apply(transform_who, axis=1)

0      0.329545
1      1.875876
2      0.304808
3      1.517143
4      0.230000
         ...   
886    0.481481
887    1.578947
888         NaN
889    1.153846
890    0.242188
Length: 891, dtype: float64

### **apply() - lambda 함수**

- 간단한 logic은 함수를 굳이 정의하지 않고, lambda 함수로 쉽게 해결

In [ ]:
df['survived'].value_counts()

survived
0    549
1    342
Name: count, dtype: int64

In [ ]:
df['survived'].apply(lambda x: '생존' if x == 1 else '사망').value_counts()

survived
사망    549
생존    342
Name: count, dtype: int64

## **Groupby()**
- 데이터를 특정 기준으로 그룹핑할 때 활용함
- groupby()를 사용할 때는 반드시 aggregate 하는 통계함수와 일반적으로 같이 적용합니다.
- 엑셀의 피봇테이블과 유사함

In [ ]:
df = sns.load_dataset('titanic')
df.groupby('sex').mean(numeric_only=True)  # 생존자와 사망자를 성별기준으로 그룹핑하여 평균 보기

,survived,pclass,age,sibsp,parch,fare,adult_male,alone
sex,,,,,,,,
female,0.742038,2.159236,27.915709,0.694268,0.649682,44.479818,0.000000,0.401274
male,0.188908,2.389948,30.726645,0.429809,0.235702,25.523893,0.930676,0.712305


In [ ]:
# 성별, 좌석등급 별 통계
# 2개 이상의 컬럼으로 그룹핑할 때도 list로 묶어서 지정하면 됨
df.groupby(['sex', 'pclass']).mean(numeric_only=True)

survived        age     sibsp     parch        fare  \
sex    pclass                                                        
female 1       0.968085  34.611765  0.553191  0.457447  106.125798   
       2       0.921053  28.722973  0.486842  0.605263   21.970121   
       3       0.500000  21.750000  0.895833  0.798611   16.118810   
male   1       0.368852  41.281386  0.311475  0.278689   67.226127   
       2       0.157407  30.740707  0.342593  0.222222   19.741782   
       3       0.135447  26.507589  0.498559  0.224784   12.661633   

               adult_male     alone  
sex    pclass                        
female 1         0.000000  0.361702  
       2         0.000000  0.421053  
       3         0.000000  0.416667  
male   1         0.975410  0.614754  
       2         0.916667  0.666667  
       3         0.919308  0.760807

1개의 특정 컬럼에 대한 결과 도출

In [ ]:
# [survived]컬럼에서 성별, 좌석등급 별 통계
df.groupby(['sex', 'pclass'])['survived'].mean()

sex     pclass
female  1         0.968085
        2         0.921053
        3         0.500000
male    1         0.368852
        2         0.157407
        3         0.135447
Name: survived, dtype: float64

In [ ]:
# DataFrame으로 출력 - pd.DataFrame()으로 감싸기
pd.DataFrame(df.groupby(['sex', 'pclass'])['survived'].mean())
# DataFrame으로 출력 - survived 컬럼을 []로 한 번 더 감싸기
df.groupby(['sex', 'pclass'])[['survived']].mean()

survived
sex    pclass          
female 1       0.968085
       2       0.921053
       3       0.500000
male   1       0.368852
       2       0.157407
       3       0.135447

### **reset_index() : 인덱스 초기화**

그룹핑된 데이터프레임의 index를 초기화하여 새로운 데이터 프레임을 생성함

In [ ]:
# index 초기화
df.groupby(['sex', 'pclass'])['survived'].mean().reset_index()

,sex,pclass,survived
0,female,1,0.968085
1,female,2,0.921053
2,female,3,0.500000
3,male,1,0.368852
4,male,2,0.157407
5,male,3,0.135447


### **다중 컬럼에 대한 결과 도출**

끝에 단일 컬럼이 아닌 여러 개의 컬럼을 지정함

In [ ]:
# 성별, 좌석등급 별 통계
df.groupby(['sex', 'pclass'])[['survived', 'age']].mean()

survived        age
sex    pclass                     
female 1       0.968085  34.611765
       2       0.921053  28.722973
       3       0.500000  21.750000
male   1       0.368852  41.281386
       2       0.157407  30.740707
       3       0.135447  26.507589

### **다중 통계 함수 적용 agg()**
- 여러 가지의 통계값 적용 시

In [ ]:
# 성별, 좌석등급 별 통계
df.groupby(['sex', 'pclass'])[['survived', 'age']].agg(['mean', 'sum'])

survived            age         
                   mean sum       mean      sum
sex    pclass                                  
female 1       0.968085  91  34.611765  2942.00
       2       0.921053  70  28.722973  2125.50
       3       0.500000  72  21.750000  2218.50
male   1       0.368852  45  41.281386  4169.42
       2       0.157407  17  30.740707  3043.33
       3       0.135447  47  26.507589  6706.42

## **Pivot_table()**
- 엑셀의 피벗과 동작이 유사함
- index, columns, values를 지정하여 피벗함

### **1개 그룹에 대한 단일 컬럼 결과**

In [ ]:
# index에 그룹을 표기
df.pivot_table(index='who', values='survived')

,survived
who,
child,0.590361
man,0.163873
woman,0.756458


In [ ]:
# columns에 그룹을 표기
df.pivot_table(columns='who', values='survived')

who,child,man,woman
survived,0.590361,0.163873,0.756458


In [ ]:
# 다중 그룹에 대한 단일 컬럼 결과
df.pivot_table(index=['who', 'pclass'], values='survived')

survived
who   pclass          
child 1       0.833333
      2       1.000000
      3       0.431034
man   1       0.352941
      2       0.080808
      3       0.119122
woman 1       0.978022
      2       0.909091
      3       0.491228

In [ ]:
# index에 컬럼을 중첩하지 않고 행과 열로 펼친 결과
df.pivot_table(index='who', columns='pclass', values='survived')

pclass,1,2,3
who,,,
child,0.833333,1.000000,0.431034
man,0.352941,0.080808,0.119122
woman,0.978022,0.909091,0.491228


In [ ]:
# 다중 통계함수 적용
df.pivot_table(index='who', columns='pclass', values='survived', aggfunc=['sum', 'mean'])

sum              mean                    
pclass   1   2   3         1         2         3
who                                             
child    5  19  25  0.833333  1.000000  0.431034
man     42   8  38  0.352941  0.080808  0.119122
woman   89  60  56  0.978022  0.909091  0.491228